In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import time

sys.path.append(os.path.join(os.getcwd(), 'src'))

from cnn import CNN, Adam
from data import DataLoader
from utils import calculate_accuracy, plot_training_history

print("✅ All libraries imported successfully!")
print("📦 Pure CNN implementation loaded from src/")


✅ All libraries imported successfully!
📦 Pure CNN implementation loaded from src/


In [2]:
# ===================================================================
# 🎛️ CONFIGURATION - Modify these settings as needed
# ===================================================================

# Training parameters - BALANCED FOR 80% ACCURACY + SPEED
EPOCHS = 8                    # Enough epochs for good learning
BATCH_SIZE = 32              # Good batch size
LEARNING_RATE = 0.003         # Balanced learning rate
IMAGE_SIZE = 28               # Small but sufficient for 80% accuracy

# Data parameters  
DATA_DIR = 'data'                      # Directory containing cat and dog folders
MAX_IMAGES_PER_CLASS = None           # Use ALL available images!

# Display settings
PLOT_DURING_TRAINING = True           # Show live training plots
SAVE_PLOTS = True                     # Save plots to files

print("🎛️ Configuration set:")
print(f"   Epochs: {EPOCHS}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Learning rate: {LEARNING_RATE}")
print(f"   Image size: {IMAGE_SIZE}x{IMAGE_SIZE}")
print(f"   Max images per class: {'ALL' if MAX_IMAGES_PER_CLASS is None else MAX_IMAGES_PER_CLASS}")
print("🎯 TARGET: 80% accuracy with reasonable speed!")



🎛️ Configuration set:
   Epochs: 8
   Batch size: 32
   Learning rate: 0.003
   Image size: 28x28
   Max images per class: ALL
🎯 TARGET: 80% accuracy with reasonable speed!


In [3]:
# ===================================================================
# 📁 Setup and Data Loading
# ===================================================================


os.makedirs('saved_models', exist_ok=True)
os.makedirs('plots', exist_ok=True)
print("✅ Created directories: saved_models/, plots/")


cats_dir = os.path.join(DATA_DIR, 'cats')
dogs_dir = os.path.join(DATA_DIR, 'dogs')

if not os.path.exists(cats_dir) or not os.path.exists(dogs_dir):
    print("❌ Data directories not found!")
    print("Please run the following in a terminal:")
    print("   python download_data.py")
    print("Or manually create:")
    print(f"   {cats_dir}/")
    print(f"   {dogs_dir}/")
    raise FileNotFoundError("Data not found")

print("✅ Data directories found!")


✅ Created directories: saved_models/, plots/
✅ Data directories found!


In [4]:

print("📁 Loading dataset...")

data_loader = DataLoader(
    cats_dir=cats_dir,
    dogs_dir=dogs_dir,
    batch_size=BATCH_SIZE,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    validation_split=0.2,
    max_images_per_class=MAX_IMAGES_PER_CLASS
)


dataset_info = data_loader.get_dataset_info()

print("\n📊 Dataset Summary:")
print(f"   Training images: {dataset_info['train_size']}")
print(f"   Validation images: {dataset_info['val_size']}")
print(f"   Image shape: {dataset_info['input_shape']}")
print(f"   Training batches: {dataset_info['train_batches']}")
print(f"   Validation batches: {dataset_info['val_batches']}")


📁 Loading dataset...
Loading cat images...
Loading dog images...
Loaded 160 training images and 40 validation images
Training: 77 cats, 83 dogs
Validation: 23 cats, 17 dogs

📊 Dataset Summary:
   Training images: 160
   Validation images: 40
   Image shape: (3, 28, 28)
   Training batches: 5
   Validation batches: 2


In [5]:
# ===================================================================
# 🧠 Create the CNN Model
# ===================================================================

print("🧠 Creating CNN model...")


model = CNN(input_shape=dataset_info['input_shape'], num_classes=1)

print(f"✅ Model created with {model.get_num_parameters():,} parameters")
print("\n🎯 OPTIMIZED Architecture (80% accuracy target):")
print("   1. Conv2D(32) → ReLU → MaxPool2D")
print("   2. Conv2D(64) → ReLU → MaxPool2D") 
print("   3. Flatten")
print("   4. Dense(32) → ReLU → Dropout(0.3)")
print("   5. Dense(1) → Sigmoid")
print("⚡ Designed for 80%+ accuracy with reasonable speed!")


optimizer = Adam(learning_rate=LEARNING_RATE)
print(f"\n⚙️ Using Adam optimizer (lr={LEARNING_RATE})")


🧠 Creating CNN model...
✅ Model created with 119,809 parameters

🎯 OPTIMIZED Architecture (80% accuracy target):
   1. Conv2D(32) → ReLU → MaxPool2D
   2. Conv2D(64) → ReLU → MaxPool2D
   3. Flatten
   4. Dense(32) → ReLU → Dropout(0.3)
   5. Dense(1) → Sigmoid
⚡ Designed for 80%+ accuracy with reasonable speed!

⚙️ Using Adam optimizer (lr=0.003)


In [6]:
# ===================================================================
# 🏋️ Training Functions
# ===================================================================

def train_epoch(model, data_loader, optimizer):
    """Train for one epoch"""
    model.train_mode()
    
    epoch_loss = 0.0
    epoch_acc = 0.0
    num_batches = 0
    
    for batch_images, batch_labels in data_loader.get_batch('train'):
        # Forward pass
        predictions = model.forward(batch_images)
        loss, dout = model.compute_loss(predictions, batch_labels)
        
        # Backward pass
        model.backward(dout)
        model.update_parameters(optimizer)
        
        # Calculate accuracy
        batch_preds = (predictions > 0.5).astype(int)
        batch_acc = calculate_accuracy(batch_preds, batch_labels)
        
        epoch_loss += loss
        epoch_acc += batch_acc
        num_batches += 1
    
    return epoch_loss / num_batches, epoch_acc / num_batches

def validate_epoch(model, data_loader):
    """Validate for one epoch"""
    model.eval_mode()
    
    epoch_loss = 0.0
    epoch_acc = 0.0
    num_batches = 0
    
    for batch_images, batch_labels in data_loader.get_batch('val'):
        predictions = model.forward(batch_images)
        loss, _ = model.compute_loss(predictions, batch_labels)
        
        batch_preds = (predictions > 0.5).astype(int)
        batch_acc = calculate_accuracy(batch_preds, batch_labels)
        
        epoch_loss += loss
        epoch_acc += batch_acc
        num_batches += 1
    
    model.train_mode()
    return epoch_loss / num_batches, epoch_acc / num_batches

print("✅ Training functions defined!")


✅ Training functions defined!


In [7]:
# ===================================================================
# 🚀 Start Training!
# ===================================================================

print("🚀 Starting training...")
print("=" * 60)


train_losses = []
val_losses = []
train_accs = []
val_accs = []
best_val_acc = 0.0


start_time = time.time()

for epoch in range(EPOCHS):
    epoch_start = time.time()
    
    print(f"\n📈 Epoch {epoch + 1}/{EPOCHS}")
    print("-" * 40)
    

    train_loss, train_acc = train_epoch(model, data_loader, optimizer)
    val_loss, val_acc = validate_epoch(model, data_loader)
    

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    

    epoch_time = time.time() - epoch_start
    

    print(f"   Train: Loss={train_loss:.4f}, Acc={train_acc:.4f}")
    print(f"   Val:   Loss={val_loss:.4f}, Acc={val_acc:.4f}")
    print(f"   Time: {epoch_time:.1f}s")
    

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        model.save_model('saved_models/best_model.pkl')
        print(f"   🎯 New best accuracy: {best_val_acc:.4f} - Model saved!")
    

    if PLOT_DURING_TRAINING and (epoch + 1) % 3 == 0:
        clear_output(wait=True)
        

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
        
        ax1.plot(train_losses, 'b-', label='Train Loss')
        ax1.plot(val_losses, 'r-', label='Val Loss')
        ax1.set_title('Training Loss')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax1.legend()
        ax1.grid(True)
        
        ax2.plot(train_accs, 'b-', label='Train Acc')
        ax2.plot(val_accs, 'r-', label='Val Acc')
        ax2.set_title('Training Accuracy')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Accuracy')
        ax2.legend()
        ax2.grid(True)
        
        plt.tight_layout()
        plt.show()
        
        print(f"📈 Epoch {epoch + 1}/{EPOCHS} completed")
        print(f"   Current: Train={train_acc:.3f}, Val={val_acc:.3f}")
        print(f"   Best Val Acc: {best_val_acc:.3f}")


total_time = time.time() - start_time
print(f"\n🎉 Training completed in {total_time:.1f} seconds!")


🚀 Starting training...

📈 Epoch 1/8
----------------------------------------
   Train: Loss=0.7538, Acc=0.5188
   Val:   Loss=0.7015, Acc=0.5469
   Time: 78.4s
Model saved to saved_models/best_model.pkl
   🎯 New best accuracy: 0.5469 - Model saved!

📈 Epoch 2/8
----------------------------------------
   Train: Loss=0.6945, Acc=0.5000
   Val:   Loss=0.7000, Acc=0.4531
   Time: 78.1s

📈 Epoch 3/8
----------------------------------------


KeyboardInterrupt: 

In [ ]:
# ===================================================================
# 📊 Final Results and Plots
# ===================================================================


model.save_model('saved_models/final_model.pkl')


if SAVE_PLOTS:
    plot_path = 'plots/training_history.png'
    plot_training_history(train_losses, val_losses, train_accs, val_accs, plot_path)


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.plot(train_losses, 'b-', label='Training Loss', linewidth=2)
ax1.plot(val_losses, 'r-', label='Validation Loss', linewidth=2)
ax1.set_title('Training and Validation Loss', fontsize=14)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(train_accs, 'b-', label='Training Accuracy', linewidth=2)
ax2.plot(val_accs, 'r-', label='Validation Accuracy', linewidth=2)
ax2.set_title('Training and Validation Accuracy', fontsize=14)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


print("=" * 60)
print("🎉 TRAINING SUMMARY")
print("=" * 60)
print(f"📈 Final Results:")
print(f"   Final train accuracy: {train_accs[-1]:.4f}")
print(f"   Final validation accuracy: {val_accs[-1]:.4f}")
print(f"   Best validation accuracy: {best_val_acc:.4f}")
print(f"\n💾 Files saved:")
print(f"   Best model: saved_models/best_model.pkl")
print(f"   Final model: saved_models/final_model.pkl")
if SAVE_PLOTS:
    print(f"   Training plot: plots/training_history.png")


In [ ]:
# ===================================================================
# 🧪 Quick Test on Validation Data
# ===================================================================

print("🧪 Testing model on validation data...")

# Get validation data
val_images, val_labels = data_loader.get_full_dataset('val')

# Test on first 20 samples
test_images = val_images[:20]
test_labels = val_labels[:20]

# Make predictions
predictions, probabilities = model.predict(test_images)

# Calculate accuracy
correct = np.sum(predictions.flatten() == test_labels.flatten())
accuracy = correct / len(test_labels)

print(f"✅ Accuracy on 20 validation samples: {correct}/20 = {accuracy:.2f}")

# Show some predictions
class_names = ['Cat', 'Dog']
print(f"\n🔍 Sample predictions:")
for i in range(min(5, len(predictions))):
    true_label = int(test_labels[i])
    pred_label = int(predictions[i])
    confidence = float(probabilities[i])
    
    if pred_label == 0:
        confidence = 1 - confidence
    
    status = "✅" if true_label == pred_label else "❌"
    print(f"   {status} True: {class_names[true_label]}, Pred: {class_names[pred_label]} ({confidence:.3f})")

print(f"\n🎯 Your model is ready for use!")
print(f"   Run the next cells to make predictions on new images!")


In [ ]:
# ===================================================================
# 🔮 Make Predictions on Sample Images
# ===================================================================

from data import ImagePreprocessor

# Create preprocessor
preprocessor = ImagePreprocessor(target_size=(IMAGE_SIZE, IMAGE_SIZE))

# Test on some images from the validation set
print("🔮 Making predictions on sample images...")

# Get a few random validation images
val_images, val_labels = data_loader.get_full_dataset('val')
sample_indices = np.random.choice(len(val_images), size=min(8, len(val_images)), replace=False)
sample_images = val_images[sample_indices]
sample_labels = val_labels[sample_indices]

# Make predictions
predictions, probabilities = model.predict(sample_images)

# Display results
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

class_names = ['Cat', 'Dog']

for i in range(len(sample_images)):
    # Convert image for display
    img = np.transpose(sample_images[i], (1, 2, 0))
    if img.max() <= 1.0:
        img = (img * 255).astype(np.uint8)
    
    axes[i].imshow(img)
    
    # Get prediction info
    true_label = int(sample_labels[i])
    pred_label = int(predictions[i])
    confidence = float(probabilities[i])
    
    if pred_label == 0:
        confidence = 1 - confidence
    
    # Create title
    title = f'True: {class_names[true_label]}\nPred: {class_names[pred_label]} ({confidence:.2f})'
    color = 'green' if true_label == pred_label else 'red'
    axes[i].set_title(title, color=color, fontsize=10)
    axes[i].axis('off')

# Hide empty subplots
for i in range(len(sample_images), 8):
    axes[i].axis('off')

plt.tight_layout()
plt.show()

print("✅ Predictions completed!")
print("🎉 Your CNN classifier is working!")
